# SnowDegreeDay-Meteorology coupling

**Goal:** Try to successfully run a coupled `SnowDegreeDay-Meteorology` simulation, with `SnowDegreeDay` as the driver.

Import the Babel-wrapped `Meteorology` and `SnowDegreeDay` components and create instances:

In [ ]:
from cmt.components import Meteorology, SnowDegreeDay
met, sno = Meteorology(), SnowDegreeDay()

Initialize the components with cfg files that, for simplicity, use the same time step and run duration:

In [ ]:
met.initialize('./input/meteorology-2.cfg')
sno.initialize('./input/snow_degree_day-1.cfg')

Get the model time step (which has been broadcast over the grid):

In [ ]:
time_step = sno.get_value('model__time_step').max()
print 'Time step = {} s'.format(time_step)

Store initial values of time and snow melt:

In [ ]:
time = [sno.get_current_time()]
snowmelt = [sno.get_value('snowpack__melt_volume_flux').max()]

Run the coupled models to completion. In each time step, perform the following actions:

1. Advance `Meteorology`
1. Get variables from `Meteorology`
1. Pass variables to `SnowDegreeDay`
1. Advance `SnowDegreeDay`

In [ ]:
until = time_step
while sno.get_current_time() < met.get_end_time():
    met.update(until)
    T_air = met.get_value('atmosphere_bottom_air__temperature')  # the only var needed by SnowDegreeDay
    
    sno.set_value('atmosphere_bottom_air__temperature', T_air)  # T_air.max() works
    sno.update(until)
    
    time.append(met.get_current_time())
    snowmelt.append(sno.get_value('snowpack__melt_volume_flux').max())
    
    until += time_step

**Error**  
Here's the relevant part of the stack trace:

```python
File "/path/to/topoflow/components/snow_base.py", line 481, in update_swe
    self.h_swe -= dh2_swe
ValueError: non-broadcastable output operand with shape () doesn't match the broadcast shape (1276,)
```

**Explanation**  
The parameter `T_air` is initialized as a scalar in Meteorology.
Therefore, in line 365 of snow_base.initialize_computed_vars(), `h_swe` is also initialized as a scalar.
However, when stepping through the models above, the Babelized get_value() method returns an array for `T_air`.
`SM` is calculated from `T_air` in line 302 of snow_degree_day.update_meltrate(), so it also becomes an array.
The error is thrown when `dh2_swe`, derived from `SM`, and therefore an array, is subtracted from `h_swe`, a scalar.

The problem occurs because the Babelized component automatically broadcasts a scalar parameter value over the entire grid. I think this is desirable behavior, but it doesn't work well with the way TopoFlow is set up.

In [ ]:
print time

In [ ]:
print snowmelt

Finalize the components:

In [ ]:
met.finalize(), sno.finalize()

**Result:** Currently failing, unless I force `T_air` to be a scalar after getting it from the Meteorology component.